In [27]:
# overlay variant information(support matrix) with scRNA matrix

In [34]:
library(Seurat)
support_mtx = Seurat::Read10X('/SGRNJ06/randd/USER/zhouyiqi/2024/test/snp1.5.1/support_mtx',gene.column = 1)

In [35]:
# convert the matrix to a dataframe
df = as.data.frame(t(support_mtx))
head(df)

,variant_1 1 114590381 A T,variant_2 1 114707695 T C,variant_3 1 114708169 T A,variant_4 1 114713817 C T,variant_5 1 114713826 C T,variant_6 1 114713846 A G,variant_7 1 114713852 A G,variant_8 1 114713859 G C,variant_9 1 114713875 A G,variant_10 1 114713880 T C,⋯,variant_197 7 55174791 T C,variant_198 7 55174796 G A,variant_199 7 55174796 G T,variant_200 7 55174797 A G,variant_201 7 55174807 A G,variant_202 7 55181309 C T,variant_203 7 55181347 A G,variant_204 7 55181370 G A,variant_205 7 55181378 C T,variant_206 7 55191822 T G
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACATCGAACAACCAACAGATTC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGCTTAAGATCGCA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGCTTAGCTAACGA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGTGATCAAGGAGC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAACGTGATTGAAGAGA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACATCGAAGACGGAGCCAAGAC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [42]:
# Pull a variant of interest
library(dplyr)
library(stringr)
chr1_114713964 = df['variant_20 1 114713964 T C']
chr1_114713964 <- chr1_114713964 %>%
  rename(`chr1:114713964:T-C` = `variant_20 1 114713964 T C`)
head(chr1_114713964)

,chr1:114713964:T-C
,<dbl>
AAACATCGAACAACCAACAGATTC,0
AAACATCGAACGCTTAAGATCGCA,1
AAACATCGAACGCTTAGCTAACGA,1
AAACATCGAACGTGATCAAGGAGC,1
AAACATCGAACGTGATTGAAGAGA,0
AAACATCGAAGACGGAGCCAAGAC,1


In [43]:
# rename number to meaningful label
chr1_114713964 <- chr1_114713964 %>%
  mutate(`chr1:114713964:T-C` = recode(as.character(`chr1:114713964:T-C`), 
                       `0` = "No Call", 
                       `1` = "ref/ref", 
                       `2` = "alt/alt", 
                       `3` = "alt/ref"))
head(chr1_114713964)

,chr1:114713964:T-C
,<chr>
AAACATCGAACAACCAACAGATTC,No Call
AAACATCGAACGCTTAAGATCGCA,ref/ref
AAACATCGAACGCTTAGCTAACGA,ref/ref
AAACATCGAACGTGATCAAGGAGC,ref/ref
AAACATCGAACGTGATTGAAGAGA,No Call
AAACATCGAAGACGGAGCCAAGAC,ref/ref


In [51]:
# merge with scRNA matrix, add the variant to metadata
seurat.data <- Read10X("/SGRNJ06/randd/USER/zhouyiqi/2024/test/snp1.5.1/scRNA_matrix")
seurat_obj <- CreateSeuratObject(seurat.data, min.cells = 1, project = "seurat_obj")
seurat_obj <- AddMetaData(object = seurat_obj, metadata = chr1_114713964)
head(seurat_obj[[]])

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


,orig.ident,nCount_RNA,nFeature_RNA,chr1:114713964:T-C
,<fct>,<dbl>,<int>,<chr>
AAACATCGAACAACCAACAGATTC,seurat_obj,786,529,No Call
AAACATCGAACGCTTAAGATCGCA,seurat_obj,415,261,ref/ref
AAACATCGAACGCTTAGCTAACGA,seurat_obj,812,460,ref/ref
AAACATCGAACGTGATCAAGGAGC,seurat_obj,597,349,ref/ref
AAACATCGAACGTGATTGAAGAGA,seurat_obj,819,576,No Call
AAACATCGAAGACGGAGCCAAGAC,seurat_obj,859,548,ref/ref
